In [2]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import os
import numpy as np

In [3]:
class HandGestureDataset(Dataset):
    def __init__(self,filepath):
        xy=np.loadtxt(filepath,delimiter=',',dtype=np.float32)
        self.len=xy.shape[0]
        self.x_data=torch.from_numpy(xy[:,:-1])
        self.y_data=torch.from_numpy(xy[:,[-1]])

    def __getitem__(self, index):
        return self.x_data[index],self.y_data[index]

    def __len__(self):
        return self.len

Training_set=HandGestureDataset('data\Training_set\Training_set.csv')
train_loader=DataLoader(dataset=Training_set,shuffle=1,num_workers=0)

Test_set=HandGestureDataset('data\Test_set\Test_set.csv')
test_loader=DataLoader(dataset=Test_set,batch_size=22,shuffle=1,num_workers=0)

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1=torch.nn.Linear(63,40)
        self.l2=torch.nn.Linear(40,20)
        self.l3=torch.nn.Linear(20,10)
        self.l4=torch.nn.Linear(10,1)
        self.sigmoid=torch.nn.Sigmoid()

    def forward(self,x):
        x=self.sigmoid(self.l1(x))
        x=self.sigmoid(self.l2(x))
        x=self.sigmoid(self.l3(x))
        return self.sigmoid(self.l4(x))

class NetTest(torch.nn.Module):
    def __init__(self):
        super(NetTest,self).__init__()
        self.l1=torch.nn.Linear(63,128)
        self.l2=torch.nn.Linear(128,64)
        self.l3=torch.nn.Linear(64,1)

        self.sigmoid=torch.nn.Sigmoid()

    def forward(self,x):
        x=self.sigmoid(self.l1(x))
        x=self.sigmoid(self.l2(x))
        x=self.sigmoid(self.l3(x))

        return x

model=NetTest()

In [5]:
criterion=torch.nn.BCELoss()
optimizer=optim.SGD(model.parameters(),lr=0.005)

In [6]:
def train(epoch):
    running_loss=0.0
    for batch_idx, data in enumerate(train_loader,0):
        inputs,target=data

        optimizer.zero_grad()

        outputs=model(inputs)
        loss=criterion(outputs,target)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()

        if batch_idx % 40==39:
            print('[%d, %5d] loss: %.3f'%(epoch+1,batch_idx+1,running_loss))
            running_loss=0

def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            inputs,labels=data
            labels=labels.long()
            
            outputs=model(inputs)
            predicted=torch.Tensor(22,1)
            predicted=outputs.data>0.5
            total+=labels.size(0)
            correct+=(predicted.long()==labels).sum().item()
    
    print('Accuracy on test set: %d %%'%(100*correct/total))
    

In [7]:
if __name__=='__main__':
    for epoch in range(150):
        train(epoch)
        test()
        

[1,    40] loss: 28.217
[1,    80] loss: 28.610
[1,   120] loss: 27.952
Accuracy on test set: 50 %
[2,    40] loss: 27.682
[2,    80] loss: 28.239
[2,   120] loss: 27.848
Accuracy on test set: 50 %
[3,    40] loss: 28.076
[3,    80] loss: 27.879
[3,   120] loss: 28.013
Accuracy on test set: 50 %
[4,    40] loss: 27.679
[4,    80] loss: 27.920
[4,   120] loss: 28.025
Accuracy on test set: 90 %
[5,    40] loss: 27.517
[5,    80] loss: 27.988
[5,   120] loss: 27.834
Accuracy on test set: 50 %
[6,    40] loss: 27.832
[6,    80] loss: 27.805
[6,   120] loss: 27.769
Accuracy on test set: 50 %
[7,    40] loss: 27.724
[7,    80] loss: 27.805
[7,   120] loss: 27.702
Accuracy on test set: 50 %
[8,    40] loss: 27.725
[8,    80] loss: 27.715
[8,   120] loss: 27.683
Accuracy on test set: 50 %
[9,    40] loss: 27.654
[9,    80] loss: 27.671
[9,   120] loss: 27.540
Accuracy on test set: 50 %
[10,    40] loss: 26.961
[10,    80] loss: 27.761
[10,   120] loss: 27.124
Accuracy on test set: 50 %
[11,   

In [8]:
torch.save(model,f='model.pkl') 

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type NetTest. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
